<a href="https://colab.research.google.com/github/kinslersi/colab/blob/main/%E7%88%AC%E8%9F%B2%E7%AC%AC%E4%BA%8C%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 判斷單個字是否為數字
def is_number(num):
  try:
    float(num)
  except ValueError:
    return False
  return True

In [ ]:
# 第一種表格樣式(大學),最後存進standard_one_list
# table: 學校進去後大表 row: 每一筆資料 department: 科系 sentence: 學測檢定標準 / 篩選倍率
def standard_one(url):        
  re=requests.get(url)
  soup=BeautifulSoup(re.text,"html.parser")
  table=soup.find('table',class_="deps-table")         
  for row in table.find_all('tr')[2:]:
    department=row.find("a",class_="name-link").text
    sentence=row.text.split("、")[-1]

    # 判斷第3,4,5是否為數字當作切分依據   
    # clear_sentence: 從sentence切分好的資料
    if not is_number(sentence[2]):
      if not is_number(sentence[3]):
        clear_sentence=sentence[4:]
      else:
        if not is_number(sentence[4]):
          clear_sentence=sentence[3:]
        else:
          clear_sentence=sentence[5:]
    else:
      clear_sentence=sentence[3:]

    if is_number(clear_sentence[0]):
      clear_sentence=clear_sentence[1:]
    elif "：" in clear_sentence:
      clear_sentence=clear_sentence.split("：")[1]
      if is_number(clear_sentence[3]):
        clear_sentence=clear_sentence[4:]

    # re1: 學測檢定標準 re2: 篩選倍率
    if " " in clear_sentence:
      re1=clear_sentence.split(" ")[0]
      re2=clear_sentence.split(" ")[1]
    elif " (" in clear_sentence:
      num=clear_sentence.index(" (")
      re1="".join(clear_sentence[:num])
      re2="".join(clear_sentence[num:])
    else:
      re1=clear_sentence
      re2="--"
    # 同學校裡每一筆處理完的資料都會放進standard_one_list
    standard_one_list.append([department,re1,re2])  

In [ ]:
# 第二種表格樣式(科大),最後存進standard_two_list
# table: 學校進去後大表 row: 每一筆資料 department: 科系 clear_sentence: 學測檢定標準 / 篩選倍率
def standard_two(url):
  re=requests.get(url)
  soup=BeautifulSoup(re.text,"html.parser")
  table=soup.find('table',class_="deps-j-table")     
  for row in table.find_all('tr')[1:]:
    deparment=row.find_all("a")[1].text
    clear_sentence=row.find_all('td')[4].text        
    standard_two_list.append([deparment,clear_sentence])

In [ ]:
# main.py
base_path="https://university-tw.ldkrsi.men/caac/"
re=requests.get(base_path)
soup=BeautifulSoup(re.text,"html.parser")

In [ ]:
# 國立學校 part
national_school=[]
container_one=soup.find_all('section',class_="container")[2]   # conainer_one: 國立學校清單 objs: 每一所學校
objs=container_one.find_all('a')
for obj in objs:                                             
    whole_path=base_path+obj['href']                           # whole path: 得到每一所學校網址
    national_school_name=obj.text.split(" ")[1]
    national_school.append(national_school_name)
    print(national_school_name+"...")

    # 兩種表格方式(國立私立都存在)，將使用表格一(stanard_one)的存在standard_one_list,使用表格二(stanard_two)的存在stardard_two_list
    standard_one_list=[]      # 每一個公私立學校一開始儲存的地方都會歸零
    standard_two_list=[]
    try:          # 匯出國立各大學csv
      standard_one(whole_path)
      df3=pd.DataFrame(standard_one_list,columns=['科系',"學測檢定標準","篩選倍率"])
      df3.to_csv(f'{national_school_name}.csv',sep=",",index=False,header=True)
    except AttributeError:   # 匯出國立各科大csv
      standard_two(whole_path)
      df4=pd.DataFrame(standard_two_list,columns=['科系','採計科目'])
      df4.to_csv(f'{national_school_name}.csv',sep=",",index=False,header=True)

國立臺灣大學...
國立臺灣師範大學...
國立中興大學...
國立成功大學...
國立政治大學...
國立清華大學...
國立陽明交通大學...
國立中央大學...
國立臺灣海洋大學...
國立高雄師範大學...
國立彰化師範大學...
國立中山大學...
國立臺北藝術大學...
國立臺中教育大學...
國立臺北教育大學...
國立臺南大學...
國立東華大學...
臺北市立大學...
國立屏東大學...
國立臺東大學...
國立體育大學...
國立中正大學...
國立臺灣藝術大學...
國立暨南國際大學...
國立臺灣體育運動大學...
國立臺南藝術大學...
國立臺北大學...
國立嘉義大學...
國立高雄大學...
國立宜蘭大學...
國立聯合大學...
國立金門大學...
國立臺灣科技大學...
國立雲林科技大學...
國立屏東科技大學...
國立臺北科技大學...
國立高雄科技大學...
國立虎尾科技大學...
國立澎湖科技大學...
國立勤益科技大學...
國立臺北護理健康大學...
國立高雄餐旅大學...
國立臺中科技大學...
國立臺北商業大學...


In [ ]:
# 私立學校 part
private_school=[]
container_two=soup.find_all('section',class_="container")[3]         # conainer_two: 私立學校清單 objs: 每一所學校
objs=container_two.find_all('li')
for obj in objs:
    whole_path=base_path+obj.find('a')['href']                      # whole path: 得到每一所學校網址
    private_school_name=obj.text.split(" ")[1]
    private_school_name=''.join([i for i in private_school_name if not i.isdigit()])
    private_school.append(private_school_name)
    print(private_school_name+"...")

    # 兩種表格方式(國立私立都存在)，將使用表格一(stanard_one)的存在standard_one_list,使用表格二(stanard_two)的存在stardard_two_list
    standard_one_list=[]          # 每一個公私立學校一開始儲存的地方都會歸零
    standard_two_list=[]
    try:        # 匯出私立各大學csv
      standard_one(whole_path)
      df5=pd.DataFrame(standard_one_list,columns=['科系',"學測檢定標準","篩選倍率"])
      df5.to_csv(f'{private_school_name}.csv',sep=",",index=False,header=True)
    except AttributeError:     # 匯出私立各科大csv
      standard_two(whole_path)
      df6=pd.DataFrame(standard_two_list,columns=['科系','採計科目'])
      df6.to_csv(f'{private_school_name}.csv',sep=",",index=False,header=True)

東吳大學...
高雄醫學大學...
中原大學...
東海大學...
中國醫藥大學...
淡江大學...
逢甲大學...
中國文化大學...
靜宜大學...
大同大學...
輔仁大學...
中山醫學大學...
長庚大學...
元智大學...
大葉大學...
中華大學...
華梵大學...
義守大學...
銘傳大學...
世新大學...
實踐大學...
長榮大學...
南華大學...
玄奘大學...
真理大學...
慈濟大學...
臺北醫學大學...
開南大學...
康寧大學...
中信金融管理學院...
佛光大學...
明道大學...
亞洲大學...
馬偕醫學院...
臺北基督學院...
朝陽科技大學...
南臺科技大學...
崑山科技大學...
嘉南藥理大學...
樹德科技大學...
龍華科技大學...
輔英科技大學...
明新科技大學...
弘光科技大學...
健行科技大學...
正修科技大學...
萬能科技大學...
建國科技大學...
明志科技大學...
高苑科技大學...
大仁科技大學...
聖約翰科技大學...
嶺東科技大學...
中國科技大學...
中臺科技大學...
台南應用科技大學...
遠東科技大學...
元培醫事科技大學...
景文科技大學...
中華醫事科技大學...
東南科技大學...
德明財經科技大學...
南開科技大學...
中華科技大學...
僑光科技大學...
育達科技大學...
美和科技大學...
吳鳳科技大學...
環球科技大學...
修平科技大學...
長庚科技大學...
敏實科技大學...
臺北城市科技大學...
醒吾科技大學...
文藻外語大學...
華夏科技大學...
慈濟科技大學...
致理科技大學...
宏國德霖科技大學...
東方設計大學...
崇右影藝科技大學...
台北海洋科技大學...
亞東科技大學...
大漢技術學院...
南亞技術學院...
黎明技術學院...
經國管理暨健康學院...


In [ ]:
# 輸出所有公私立學校清單
df1=pd.DataFrame({'school':national_school},index=[i for i in range(1,len(national_school)+1)])
df2=pd.DataFrame({'school':private_school},index=[i for i in range(1,len(private_school)+1)])
df=pd.concat([df1,df2],keys=['國立學校','私立學校'])
print(df)
df.to_csv('學校清單.csv',sep=",")   
print('done')

            school
國立學校 1      國立臺灣大學
     2    國立臺灣師範大學
     3      國立中興大學
     4      國立成功大學
     5      國立政治大學
...            ...
私立學校 83     亞東科技大學
     84     大漢技術學院
     85     南亞技術學院
     86     黎明技術學院
     87  經國管理暨健康學院

[131 rows x 1 columns]
done
